In [4]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [5]:
import os 
import tensorflow as tf
from pathlib import Path

In [6]:
import cv2

In [7]:
# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:03:00.0, compute capability: 7.5



In [8]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet
import numpy as np

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam, SGD
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, SINGLE_CLASS_METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, BinaryCrossentropy, compute_class_weight

In [9]:
#CHEXPERT_BENCHMARKS, CHESTXRAY14_BENCHMARKS = generate_benchmarks (batch_sizes={"Medium": 32}, epoch_sizes= {"Dev": 5,}, crop={"C0": False}, name_suffix="",)



In [10]:
#CHEXPERT_BENCHMARKS.keys() 

In [11]:
#CHESTXRAY14_BENCHMARKS.keys()

In [12]:
#[m.name for m in METRICS]

In [13]:
#CHEXPERT_COLUMNS

In [14]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('tensorboard', 'http://e557232b7b6b.ngrok.io'), ('jupyterlab', 'http://75243960621c.ngrok.io')]


In [15]:
#CHEXPERT_BENCHMARKS["BCE_E5_B32_C0"].as_dict()

In [16]:
#chexpert_exp = simple_architecture_experiment(CHEXPERT_BENCHMARKS["BCE_E5_B32_C0"], DenseNet121, CHEXPERT_COLUMNS)

In [17]:
#chexpert_exp.benchmark.as_dict()

In [18]:
#chexpert_exp.run()

In [19]:
columns_12 = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']

uzeros = ['Cardiomegaly',
        'Enlarged Cardiomediastinum',
        'Lung Opacity',
        'Lung Lesion',
        'Consolidation',
        'Pneumothorax',
        'Pleural Effusion']

uones = ['Edema',
        'Atelectasis',
        'Fracture',
        'Pleural Other',
        'Pneumonia',]


upsample_factors = {
    "Enlarged Cardiomediastinum": 1,
    "Lung Lesion":1,
    #"Pneumothorax":1,
    #"Pneumonia":1,
    "Pleural Other":2,
    "Fracture":2,
}

columns_5 =  ['Cardiomegaly',
                'Edema',
                'Consolidation',
                'Atelectasis',
                'Pleural Effusion']

uzeros_5 = ['Cardiomegaly',
        'Consolidation',
        'Pleural Effusion']

uones_5 = ['Edema',
        'Atelectasis']


upsample_factors_5 = {
    "Consolidation":2,
    "Cardiomegaly":1
}

transformations_0 = {"hist_equalization":{}}
transformations_1 = { "gaussian_blur":{"kernal_size":3}, "hist_equalization":{}}
transformations_2 = {"unsharp_mask":{"radius":2, "amount":1}}
transformations_3 = {"windowing"}

In [20]:
bce_benchmark = Chexpert_Benchmark (path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                 name="Chexpert_BCE_E3_B32_C0_N12_AugAffine_Uones_D256_DS9505_2LR1_LF5_SGD_Upsampled",
                                 classes=columns_12,
                                 train_labels = "train.csv",
                                 test_labels = "test.csv",
                                 nan_replacement = 0, #float("NaN"),
                                 u_enc = "uones",
                                 epochs=3,
                                 batch_size=32,
                                 crop = False,
                                 dim=(256, 256),
                                 loss = BinaryCrossentropy(),
                                 use_class_weights = False,
                                 upsample_factors = upsample_factors,
                                 metrics=METRICS,
                                 single_class_metrics=SINGLE_CLASS_METRICS,
                                 optimizer = SGD(learning_rate=2e-1, clipnorm=1),
                                 lr_factor = 0.5,
                                 augmentation = "affine",
                                 transformations = {},
                                 split_seed = 6122156,
                                 split_valid_size = 0.05, 
                                 preprocess_input_fn = tf.keras.applications.densenet.preprocess_input)

#bce_benchmark.loss = WeightedBinaryCrossentropy(bce_benchmark.positive_weights,
 #                                               bce_benchmark.negative_weights)

Initialzed custom_binary_crossentropy with epsilon 1e-05


In [20]:
bce_benchmark.as_dict()

{'benchmark_name': 'Chexpert_BCE_E3_B32_C0_N12_AugAffine_Uones_D256_DS9505_2LR1_LF5_SGD_Upsampled',
 'dataset_name': 'chexpert_full',
 'dataset_folder': 'data/chexpert/full',
 'models_dir': 'models',
 'epochs': 3,
 'optimizer': 'SGD',
 'learning_rate': 0.2,
 'lr_factor': 0.5,
 'loss': 'custom_binary_crossentropy',
 'use_class_weights': False,
 'class_weights': [0.6595874428749084,
  0.877420961856842,
  0.2572748064994812,
  1.3363227844238281,
  0.49572181701660156,
  0.692219078540802,
  1.1935983896255493,
  0.44563019275665283,
  1.27659273147583,
  0.2977822422981262,
  1.450164556503296,
  1.017684817314148],
 'positive_weights': [0.6595874428749084,
  0.877420961856842,
  0.2572748064994812,
  1.3363227844238281,
  0.49572181701660156,
  0.692219078540802,
  1.1935983896255493,
  0.44563019275665283,
  1.27659273147583,
  0.2977822422981262,
  1.450164556503296,
  1.017684817314148],
 'negative_weights': [0.7886719107627869,
  0.7436374425888062,
  1.275024175643921,
  0.7019586

In [21]:
bce_chexpert_exp = simple_architecture_experiment(bce_benchmark, DenseNet121, bce_benchmark.label_columns)

In [22]:
bce_chexpert_exp.train()

Epoch 1/3
Some predictions are greater than 1
Some predictions are less than 0
Some predictions are greater than 1
Some predictions are less than 0
8868/8868 [==============================] - ETA: 0s - loss: 0.4165 - auc: 0.7095 - precision: 0.6455 - recall: 0.3260 - f2_score: 0.3617 - binary_accuracy: 0.8196 - accuracy_enlarged_cardiomediastinum: 0.8054 - accuracy_cardiomegaly: 0.8622 - accuracy_lung_opacity: 0.6383 - accuracy_lung_lesion: 0.9028 - accuracy_edema: 0.7824 - accuracy_consolidation: 0.8124 - accuracy_pneumonia: 0.8902 - accuracy_atelectasis: 0.7072 - accuracy_pneumothorax: 0.9004 - accuracy_pleural_effusion: 0.7496 - accuracy_pleural_other: 0.9106 - accuracy_fracture: 0.8735 - auc_enlarged_cardiomediastinum: 0.5828 - auc_cardiomegaly: 0.7661 - auc_lung_opacity: 0.6849 - auc_lung_lesion: 0.6989 - auc_edema: 0.8091 - auc_consolidation: 0.6481 - auc_pneumonia: 0.6708 - auc_atelectasis: 0.6723 - auc_pneumothorax: 0.7212 - auc_pleural_effusion: 0.8189 - auc_pleural_other: 0.

In [22]:
bce_chexpert_exp.model.load_weights("models/DenseNet121_Chexpert_BCE_E3_B32_C0_N12_AugAffine_Uones_D256_DS9505_2LR1_LF5_SGD_Upsampled/weights.03-0.35.hdf5")
bce_chexpert_exp.evaluate()

234/234 [==============================] - 4s 17ms/step


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Some predictions are greater than 1
Some predictions are less than 0
Some predictions are greater than 1
Some predictions are less than 0
234/234 [==============================] - 14s 58ms/step - loss: 0.3289 - auc: 0.7735 - precision: 0.7961 - recall: 0.3718 - f2_score: 0.4162 - binary_accuracy: 0.8593 - accuracy_enlarged_cardiomediastinum: 0.5385 - accuracy_cardiomegaly: 0.7735 - accuracy_lung_opacity: 0.8205 - accuracy_lung_lesion: 0.9829 - accuracy_edema: 0.8846 - accuracy_consolidation: 0.8632 - accuracy_pneumonia: 0.9615 - accuracy_atelectasis: 0.6838 - accuracy_pneumothorax: 0.9615 - accuracy_pleural_effusion: 0.8803 - accuracy_pleural_other: 0.9744 - accuracy_fracture: 0.9872 - auc_enlarged_cardiomediastinum: 0.5335 - auc_cardiomegaly: 0.8003 - auc_lung_opacity: 0.9259 - auc_lung_lesion: 0.9227 - auc_edema: 0.9276 - auc_consolidation: 0.8826 - auc_pneumonia: 0.8144 - auc_atelectasis: 0.8281 - auc_pneumothorax: 0.8081 - auc_pleural_effusion: 0.9141 - auc_pleural_other: 0.9249 -

{'report': '                            precision    recall  f1-score   support\n\nEnlarged Cardiomediastinum       0.60      0.03      0.05       109\n              Cardiomegaly       0.89      0.25      0.39        68\n              Lung Opacity       0.93      0.72      0.81       126\n               Lung Lesion       0.00      0.00      0.00         1\n                     Edema       0.70      0.69      0.70        45\n             Consolidation       1.00      0.03      0.06        33\n                 Pneumonia       0.33      0.12      0.18         8\n               Atelectasis       0.67      0.15      0.24        80\n              Pneumothorax       0.33      0.12      0.18         8\n          Pleural Effusion       0.87      0.69      0.77        67\n             Pleural Other       0.00      0.00      0.00         1\n                  Fracture       0.00      0.00      0.00         0\n\n                 micro avg       0.80      0.37      0.51       546\n                 m

In [34]:
bce_chexpert_exp.save()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /srv/idp-radio-1/models/DenseNet121_Chexpert_BCE_E3_B32_C0_N12_AugAffine_Uones_D256_DS9505_2LR1_LF5_SGD_Upsampled/DenseNet121_Chexpert_BCE_E3_B32_C0_N12_AugAffine_Uones_D256_DS9505_2LR1_LF5_SGD_Upsampled_20200904-092128/assets


'03479e56-ee90-11ea-b021-0242ac110002'

In [18]:
bce_benchmark = Chexpert_Benchmark (path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                 name="Chexpert_BCE_E3_B32_C0_N12_NoAug_UZEROES_D256_DS9505_2LR1_LF5_SGD_Upsampled12",
                                 classes=columns_12,
                                 train_labels = "train.csv",
                                 test_labels = "test.csv",
                                 nan_replacement = 0, #float("NaN"),
                                 u_enc = "uzeroes",
                                 epochs=3,
                                 batch_size=32,
                                 crop = False,
                                 dim=(256, 256),
                                 loss = BinaryCrossentropy(),
                                 use_class_weights = False,
                                 upsample_factors = upsample_factors,
                                 metrics=METRICS,
                                 single_class_metrics=SINGLE_CLASS_METRICS,
                                 optimizer = SGD(learning_rate=2e-1, clipnorm=1),
                                 lr_factor = 0.5,
                                 #augmentation = "affine",
                                 transformations = {},
                                 split_seed = 6122156,
                                 split_valid_size = 0.05, 
                                 preprocess_input_fn = tf.keras.applications.densenet.preprocess_input)


Initialzed custom_binary_crossentropy with epsilon 1e-05


In [19]:
bce_chexpert_exp = simple_architecture_experiment(bce_benchmark, DenseNet121, bce_benchmark.label_columns)

In [ ]:
bce_chexpert_exp.train()

Epoch 1/3
Some predictions are greater than 1
Some predictions are less than 0
Some predictions are greater than 1
Some predictions are less than 0
2996/8101 [==========>...................] - ETA: 20:58 - loss: 0.3366 - auc: 0.6917 - precision: 0.6224 - recall: 0.3221 - f2_score: 0.3564 - binary_accuracy: 0.8656 - accuracy_enlarged_cardiomediastinum: 0.9023 - accuracy_cardiomegaly: 0.8841 - accuracy_lung_opacity: 0.6294 - accuracy_lung_lesion: 0.9166 - accuracy_edema: 0.8014 - accuracy_consolidation: 0.9352 - accuracy_pneumonia: 0.9726 - accuracy_atelectasis: 0.8517 - accuracy_pneumothorax: 0.9127 - accuracy_pleural_effusion: 0.7478 - accuracy_pleural_other: 0.9477 - accuracy_fracture: 0.8841 - auc_enlarged_cardiomediastinum: 0.5880 - auc_cardiomegaly: 0.7365 - auc_lung_opacity: 0.6738 - auc_lung_lesion: 0.6683 - auc_edema: 0.7904 - auc_consolidation: 0.6584 - auc_pneumonia: 0.6402 - auc_atelectasis: 0.6191 - auc_pneumothorax: 0.7102 - auc_pleural_effusion: 0.8134 - auc_pleural_other:

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



8101/8101 [==============================] - ETA: 0s - loss: 0.3242 - auc: 0.7239 - precision: 0.6415 - recall: 0.3516 - f2_score: 0.3864 - binary_accuracy: 0.8701 - accuracy_enlarged_cardiomediastinum: 0.9037 - accuracy_cardiomegaly: 0.8881 - accuracy_lung_opacity: 0.6430 - accuracy_lung_lesion: 0.9176 - accuracy_edema: 0.8085 - accuracy_consolidation: 0.9349 - accuracy_pneumonia: 0.9731 - accuracy_atelectasis: 0.8536 - accuracy_pneumothorax: 0.9145 - accuracy_pleural_effusion: 0.7702 - accuracy_pleural_other: 0.9482 - accuracy_fracture: 0.8850 - auc_enlarged_cardiomediastinum: 0.6255 - auc_cardiomegaly: 0.7825 - auc_lung_opacity: 0.6933 - auc_lung_lesion: 0.7062 - auc_edema: 0.8118 - auc_consolidation: 0.6837 - auc_pneumonia: 0.6699 - auc_atelectasis: 0.6461 - auc_pneumothorax: 0.7577 - auc_pleural_effusion: 0.8406 - auc_pleural_other: 0.7619 - auc_fracture: 0.6968 - precision_enlarged_cardiomediastinum: 0.4517 - precision_cardiomegaly: 0.5788 - precision_lung_opacity: 0.6186 - preci

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6362/8101 [======================>.......] - ETA: 6:46 - loss: 0.2850 - auc: 0.8122 - precision: 0.6856 - recall: 0.4440 - f2_score: 0.4777 - binary_accuracy: 0.8830 - accuracy_enlarged_cardiomediastinum: 0.9073 - accuracy_cardiomegaly: 0.8992 - accuracy_lung_opacity: 0.6748 - accuracy_lung_lesion: 0.9249 - accuracy_edema: 0.8291 - accuracy_consolidation: 0.9346 - accuracy_pneumonia: 0.9735 - accuracy_atelectasis: 0.8537 - accuracy_pneumothorax: 0.9225 - accuracy_pleural_effusion: 0.8159 - accuracy_pleural_other: 0.9578 - accuracy_fracture: 0.9026 - auc_enlarged_cardiomediastinum: 0.7386 - auc_cardiomegaly: 0.8616 - auc_lung_opacity: 0.7389 - auc_lung_lesion: 0.8195 - auc_edema: 0.8608 - auc_consolidation: 0.7449 - auc_pneumonia: 0.7545 - auc_atelectasis: 0.7139 - auc_pneumothorax: 0.8665 - auc_pleural_effusion: 0.8914 - auc_pleural_other: 0.9011 - auc_fracture: 0.8551 - precision_enlarged_cardiomediastinum: 0.7007 - precision_cardiomegaly: 0.6582 - precision_lung_opacity: 0.6477 - pre

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



3854/8101 [=============>................] - ETA: 16:32 - loss: 0.2550 - auc: 0.8567 - precision: 0.7179 - recall: 0.5147 - f2_score: 0.5455 - binary_accuracy: 0.8939 - accuracy_enlarged_cardiomediastinum: 0.9175 - accuracy_cardiomegaly: 0.9040 - accuracy_lung_opacity: 0.6931 - accuracy_lung_lesion: 0.9364 - accuracy_edema: 0.8371 - accuracy_consolidation: 0.9350 - accuracy_pneumonia: 0.9732 - accuracy_atelectasis: 0.8554 - accuracy_pneumothorax: 0.9330 - accuracy_pleural_effusion: 0.8284 - accuracy_pleural_other: 0.9762 - accuracy_fracture: 0.9379 - auc_enlarged_cardiomediastinum: 0.8156 - auc_cardiomegaly: 0.8819 - auc_lung_opacity: 0.7623 - auc_lung_lesion: 0.8929 - auc_edema: 0.8768 - auc_consolidation: 0.7790 - auc_pneumonia: 0.7997 - auc_atelectasis: 0.7451 - auc_pneumothorax: 0.9056 - auc_pleural_effusion: 0.9042 - auc_pleural_other: 0.9703 - auc_fracture: 0.9475 - precision_enlarged_cardiomediastinum: 0.8006 - precision_cardiomegaly: 0.6815 - precision_lung_opacity: 0.6658 - pr

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7638/8101 [===========================>..] - ETA: 1:48 - loss: 0.2533 - auc: 0.8584 - precision: 0.7190 - recall: 0.5189 - f2_score: 0.5494 - binary_accuracy: 0.8946 - accuracy_enlarged_cardiomediastinum: 0.9181 - accuracy_cardiomegaly: 0.9045 - accuracy_lung_opacity: 0.6949 - accuracy_lung_lesion: 0.9374 - accuracy_edema: 0.8373 - accuracy_consolidation: 0.9353 - accuracy_pneumonia: 0.9733 - accuracy_atelectasis: 0.8557 - accuracy_pneumothorax: 0.9335 - accuracy_pleural_effusion: 0.8285 - accuracy_pleural_other: 0.9767 - accuracy_fracture: 0.9405 - auc_enlarged_cardiomediastinum: 0.8214 - auc_cardiomegaly: 0.8824 - auc_lung_opacity: 0.7645 - auc_lung_lesion: 0.8948 - auc_edema: 0.8767 - auc_consolidation: 0.7795 - auc_pneumonia: 0.7993 - auc_atelectasis: 0.7455 - auc_pneumothorax: 0.9077 - auc_pleural_effusion: 0.9045 - auc_pleural_other: 0.9726 - auc_fracture: 0.9517 - precision_enlarged_cardiomediastinum: 0.7945 - precision_cardiomegaly: 0.6817 - precision_lung_opacity: 0.6677 - pre

In [ ]:
bce_chexpert_exp.evaluate()

In [ ]:
bce_chexpert_exp.save()

In [ ]:
compute_class_weight(bce_chexpert_exp.benchmark.traingen)

In [ ]:
trained_weights = bce_chexpert_exp.model.get_weights()

In [ ]:
images, labels = bce_chexpert_exp.benchmark.traingen[0]
len(images)

In [ ]:
preds = bce_chexpert_exp.model(images, training=True)
preds.op

In [ ]:
bce_chexpert_exp.model.summary(line_length=150)

In [ ]:
index = -1
trained_weights[index].shape, trained_weights[index], np.isnan(trained_weights[index]).any()

In [ ]:
nan_layers = [i for i in range(len(trained_weights)) if np.isnan(trained_weights[i]).any()] 
#nan_layers   

In [ ]:
mean_weights = {i:np.abs(trained_weights[i]).mean() for i in range(len(trained_weights))}
#mean_weights

In [ ]:
{i:trained_weights[i].shape for i in range(len(trained_weights))}

In [ ]:
for i in range(191, 195):
    print(i, trained_weights[i].shape, trained_weights[i])

In [ ]:
#chexpert_exp.save()

In [ ]:
from datetime import datetime
model_filename = chexpert_exp.model_name + "_" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_filename_tf = model_filename + ".tf"
model_filename_h5 = model_filename + ".h5"
model_filename

In [ ]:
folderpath = Path(os.getcwd()) / 'models' / chexpert_exp.model_name
path = folderpath / model_filename
path

In [ ]:
# make sure path exists, ceate one if necessary
Path(folderpath).mkdir(parents=True, exist_ok=True)
chexpert_exp.model.save(path, save_format="tf")

In [ ]:
# make sure path exists, ceate one if necessary
Path(folderpath).mkdir(parents=True, exist_ok=True)
chexpert_exp.model.save(folderpath / model_filename_h5, save_format="h5")

In [ ]:
!pwd

In [ ]:
folderpath / model_filename_h5

In [ ]:
chexpert_exp.save()

In [ ]:
chexpert_exp.evaluate()

In [ ]:
testgen = chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].testgen

In [ ]:
preds = chexpert_exp.model.predict(testgen, steps=len(testgen), verbose=1)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import models
import json
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from pathlib import Path
from keras.utils.generic_utils import get_custom_objects
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight


groundtruth_label = testgen.get_labels_nonan()
thresholds = [0.5]
for t in thresholds:
    print('current threshold: ', t)
    predictions_bool = (preds >= t)
    y_pred = np.array(predictions_bool, dtype=int)

    # report = classification_report(groundtruth_label, y_pred, target_names=list(CHEXPERT_BENCHMARKS['WBCE_E5_B32_C0'].label_columns))
    # print('sklearn report: ', report)
    
    pre = Precision(thresholds=t)
    pre.update_state(groundtruth_label, predictions_bool)
    print('TF Precision: ', pre.result())
    pre.reset_states()
    
    rec = Recall(thresholds=t)
    rec.update_state(groundtruth_label, predictions_bool)
    print('TF Recall: ', rec.result())
    rec.reset_states()

In [ ]:
report = classification_report(groundtruth_label, y_pred, target_names=chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].label_columns)
print('sklearn report: ', report)

In [ ]:
model = tf.keras.models.load_model('models/DenseNet121_Chexpert_BCE_E1_B32_C0_N5_D256_DS0595_savetest7/DenseNet121_Chexpert_BCE_E1_B32_C0_N5_D256_DS0595_savetest7_20200819-125620.h5', compile=False)

In [ ]:
predstwo = model.predict(testgen, steps=len(testgen), verbose=1)

In [ ]:
groundtruth_label = testgen.get_labels_nonan()
predictions_bool = (predstwo >= t)
y_pred = np.array(predictions_bool, dtype=int)
report = classification_report(groundtruth_label, y_pred, target_names=chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].label_columns)
print('sklearn report: ', report)

In [ ]:
"test.h5".replace(".h5", "")

In [ ]:
# model_guendel_chestxray14 = densenet(classes=len(CHESTXRAY14_COLUMNS))
model_guendel_chexpert = densenet(classes=len(CHEXPERT_COLUMNS))

#experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14)
experiment_guendel_chexpert = Experiment(CHEXPERT_BENCHMARKS["WBCE_E10_B32"], model_guendel_chexpert, model_name="test_WBCE_32")



In [ ]:
#experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()

In [ ]:
experiment_guendel_chexpert.evaluate()

In [ ]:
experiment_guendel_chexpert.save(upload=False)

In [ ]:
print("done")

In [ ]:
CHEXPERT_BENCHMARKS["WBCE_E10_B32"].as_dict()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import Loss
from tensorflow.python.ops.losses import util as tf_losses_util
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import constant_op
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.keras import backend_config

In [ ]:
weights_pos = tf.ones((4)) * 20.0
weights_neg = tf.ones((4)) * 0.5
y_true = tf.constant([[0, -1, -1, 1], [-1, 0, -1, -1]], dtype=tf.float32)
y_pred = tf.constant([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]])
loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
loss

In [ ]:
y_true = tf.constant(y_true, dtype=tf.float32)
y_true

In [ ]:
y_pred = tf.constant(y_pred)
y_pred

In [ ]:
if tensor_util.is_tensor(y_pred) and tensor_util.is_tensor(y_true):
    y_pred, y_true = tf_losses_util.squeeze_or_expand_dimensions(
        y_pred, y_true)

epsilon_ = constant_op.constant(
     backend_config.epsilon(), dtype=y_pred.dtype.base_dtype)
y_pred = clip_ops.clip_by_value(y_pred, epsilon_, 1. - epsilon_)

In [ ]:
mask = tf.cast(tf.math.greater_equal(y_true, 0), tf.float32)

# Compute cross entropy from probabilities.
bce_pos = y_true * math_ops.log(y_pred + epsilon_)
bce_neg = (1 - y_true) * math_ops.log(1 - y_pred + epsilon_)

bce_pos = tf.math.multiply_no_nan(bce_pos, mask)
bce_neg = tf.math.multiply_no_nan(bce_neg, mask)

bce = bce_pos * weights_pos + \
    bce_neg * weights_neg

loss = tf.reduce_sum(-bce) / tf.reduce_sum(mask)
loss

In [ ]:
tf.reduce_sum(mask)

In [ ]:
bce_neg

In [ ]:
bce

In [ ]:

from tensorflow.keras import backend as K
